In [1]:
import numpy as np
import sys
sys.path.append("../")
import fh_comm as fhc
from scipy.sparse.linalg import norm
max_level = 3

# Test the commutator of basic bosonic operators
The Basic bosonic operators:
$$Ba_i = b^{\dagger}_i+b_i$$
$$Bm_i = b^{\dagger}_i-b_i$$
$$Bn_i = b^{\dagger}_ib_i$$
Theoretically, we could obtain the flowing result:
$$[Ba,Bm] = 2\text{I}$$
$$[Ba,Bn] = -Bm$$
$$[Bm,Bn] = -Ba$$

In [2]:
Ba = fhc.BosonAddOp((0,),1.,max_level)
Bm = fhc.BosonMinuOp((0,),1.,max_level)
Bn = fhc.BosonNumOp((0,),1.,max_level)

print('The result of [Ba,Bm]:')
print(fhc.commutator(Ba,Bm))
print('The result of [Ba,Bn]:')
print(fhc.commutator(Ba,Bn))
print('The result of [Bm,Bn]:')
print(fhc.commutator(Bm,Bn))

The result of [Ba,Bm]:
(2.0) <empty product>
The result of [Ba,Bn]:
(-1.0) Bn_(0,)
The result of [Bm,Bn]:
(-1.0) Ba_(0,)


# compute the result of decomposed hamiltonian 1D

In [3]:
v = -1
u = 1
w0 = 1
g = 1
mod = 1
max_level=3
translatt = fhc.SubLattice(np.array([[2]]))
h0 = fhc.SumOp([fhc.HoppingOp(( 0,), ( 1,), s, v,mod,max_level) for s in [0, 1]])
h1 = fhc.SumOp([fhc.HoppingOp((-1,), ( 0,), s, v,mod,max_level) for s in [0, 1]])
h2 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp((x,), s, 1,mod,max_level) for s in [0, 1]], u) for x in [0, 1]])
h3 = fhc.SumOp([fhc.BosonNumOp((x,), w0, max_level=max_level) for x in [0,1]])
h4 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp((x,),s,1.,mod,max_level), fhc.BosonAddOp((x,),1.,max_level)],g) for s in [0,1] for x in [0,1]])
hlist = [h0,h1,h2,h3,h4]

In [4]:
print('the result of [h4,h2]:')
comm_4_2 = fhc.commutator_translation(h4,h2,translatt=translatt)
print(comm_4_2)
print('the result of [h4,h3]:')
comm_4_3 = fhc.commutator_translation(h4,h3,translatt=translatt)
print(comm_4_3)

the result of [h4,h2]:
<zero op>
the result of [h4,h3]:
(n_{(0,), up}) @ ((-1.0) Bn_(0,)) + (n_{(1,), up}) @ ((-1.0) Bn_(1,)) + (n_{(0,), dn}) @ ((-1.0) Bn_(0,)) + (n_{(1,), dn}) @ ((-1.0) Bn_(1,))


In [5]:
print('the result of [h3,[h4,h3]]:')
comm_3_4_3 = fhc.simplify(fhc.commutator_translation(h3,comm_4_3,translatt=translatt))
print(comm_3_4_3)
print('the result of [h4,[h4,h3]]:')
comm_4_4_3 = fhc.simplify(fhc.commutator_translation(h4,comm_4_3,translatt=translatt))
print(comm_4_4_3)

the result of [h3,[h4,h3]]:
(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,)) + (-1.0) (n_{(1,), up} + n_{(1,), dn}) @ (Ba_(1,))
the result of [h4,[h4,h3]]:
(-2.0) n_{(0,), up} + (-2.0) n_{(1,), up} + (-2.0) n_{(0,), dn} + (-2.0) n_{(1,), dn} + (-4.0) (n_{(0,), up}) @ (n_{(0,), dn}) + (-4.0) (n_{(1,), up}) @ (n_{(1,), dn})


# Get the error bound of commutator

In [6]:
eb_3_4_3 = comm_3_4_3.norm_bound()
eb_4_4_3 = fhc.simplify(comm_4_4_3).norm_bound()
print(eb_3_4_3)
print(eb_4_4_3)

9.33765687335591
16.0


Theoretically, $[h_j,[h4,h3]]=0$, where $j=0,1,2$

Here we test the commutator table to varify the result

In [10]:
comm_tab = fhc.NestedCommutatorTable(hlist, 3, translatt)
tab2 = comm_tab.table(2)
print("tab2[1][4][3]:", tab2[1][4][3])
print("tab2[1][4][3].norm_bound():", tab2[1][4][3].norm_bound())

print("tab2[3][4][3]:", tab2[3][4][3])
print("tab2[3][4][3].norm_bound():", tab2[3][4][3].norm_bound())

tab2[1][4][3]: (-1.0) (g_{(-1,), (0,), up}) @ (Bn_(-1,)) + (g_{(-1,), (0,), up}) @ (Bn_(0,)) + (-1.0) (g_{(-1,), (0,), dn}) @ (Bn_(-1,)) + (g_{(-1,), (0,), dn}) @ (Bn_(0,))
tab2[1][4][3].norm_bound(): 6.928203230275509
tab2[3][4][3]: (-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,)) + (-1.0) (n_{(1,), up} + n_{(1,), dn}) @ (Ba_(1,))
tab2[3][4][3].norm_bound(): 6.928203230275498


In [7]:
print(tab2[3][4][3].terms[0])
print(comm_3_4_3.terms[0])
tab2[3][4][3].terms[0].proportional(comm_3_4_3.terms[0])

(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,))
(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,))


True

In [8]:
[[op1,op2] for op1, op2 in zip(tab2[3][4][3].terms[0].ops, comm_3_4_3.terms[0].ops)]

[[<fh_comm.hamiltonian_ops.SumOp at 0x1fcdbf16c50>,

Then we compute the err_bound

In [12]:
print("bound specialized for second-order Suzuki method:")
comm_bound_terms = fhc.commutator_bound_strang(len(hlist))

def get_num_int(idx):
    num_int=0
    for i in idx:
        if i==2:
            num_int+=1
        elif i==3:
            num_int+=3
        elif i==4:
            num_int+=8
    return num_int

# sort by number of interaction terms
err_bound = 20 * [0]
for term in comm_bound_terms:
    #print(term)
    # number of occurrences of interaction term in current nested commutator
    num_int = get_num_int(term.commidx)
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
# the entries still need to be divided by 2 to obtain the error per lattice site
print("err_bound:", err_bound)

bound specialized for second-order Suzuki method:
err_bound: [1.0, 1.3333333333333206, 0.3333333333333333, Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), 3.4641016151376878, 2.3094010767585034, Fraction(0, 1), 1.1547005383792515, Fraction(0, 1), 0, 0.28867513459481287, 0, 4.0, Fraction(0, 1), 0, 1.3333333333333333]


In [4]:
# to large

comm_tab = fhc.NestedCommutatorTable(hlist, 5, translatt)
tab4 = comm_tab.table(4)
method = fhc.SplittingMethod.suzuki(len(hlist), 2)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound(method,s)
err_bound = 90*[0]
def get_num_int(idx):
    num_int=0
    for i in idx:
        if i==2:
            num_int+=1
        elif i==3:
            num_int+=5
        elif i==4:
            num_int+=22
    return num_int
for term in comm_bound_terms:
    #print(term)
    # number of occurrences of interaction term in current nested commutator
    num_int = get_num_int(term.commidx)
    err_bound[num_int] += term.weight * tab4[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()
# the entries still need to be divided by 2 to obtain the error per lattice site
print("err_bound:", err_bound)

MemoryError: Unable to allocate 40.5 GiB for an array with shape (5435817984,) and data type float64

Test for 2D case

In [2]:
v = -1
u = 1
w0 = 1
g = 1
mod = 1
max_level=3
plaqcoords = [( 0,  0), ( 1,  0), ( 1,  1), ( 0,  1)]
plaqcshift = [(-1, -1), ( 0, -1), ( 0,  0), (-1,  0)]
translatt = fhc.SubLattice(np.array([[2, 0], [0, 2]]))
h0 = fhc.SumOp([fhc.HoppingOp(plaqcoords[i], plaqcoords[(i + 1) % 4], s, v,mod,max_level) for s in [0, 1] for i in range(4)])
h1 = fhc.SumOp([fhc.HoppingOp(plaqcshift[i], plaqcshift[(i + 1) % 4], s, v,mod,max_level) for s in [0, 1] for i in range(4)])
h2 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp(plaqcoords[i], s, 1,mod,max_level) for s in [0, 1]], u,mod,max_level) for i in range(4)])
h3 = fhc.SumOp([fhc.BosonNumOp(plaqcoords[i], w0, max_level=max_level) for i in range(4)])
h4 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp(plaqcoords[i],s,1.,mod,max_level), fhc.BosonAddOp(plaqcoords[i],1.,max_level)],g) for s in [0,1] for i in range(4)])
hlist = [h0,h1,h2,h3,h4]

In [3]:
comm_tab = fhc.NestedCommutatorTable(hlist, 3, translatt)
tab2 = comm_tab.table(2)

In [4]:
print("bound specialized for second-order Suzuki method:")
comm_bound_terms = fhc.commutator_bound_strang(len(hlist))

def get_num_int(idx):
    num_int=0
    for i in idx:
        if i==2:
            num_int+=1
        elif i==3:
            num_int+=3
        elif i==4:
            num_int+=8
    return num_int

# sort by number of interaction terms
err_bound = 20 * [0]
for term in comm_bound_terms:
    #print(term)
    # number of occurrences of interaction term in current nested commutator
    num_int = get_num_int(term.commidx)
    print(term.commidx)
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
# the entries still need to be divided by 2 to obtain the error per lattice site
print("err_bound:", err_bound)

bound specialized for second-order Suzuki method:
(0, 1, 0)


In [ ]:
from scipy import sparse
U = sparse.csr_matrix([[ 0.,  0.], [ 1.,  0.]])
c = sparse.identity(1,format='csr')
for i in range(10):
    print(c)
    c = sparse.kron(c,U)
    c.eliminate_zeros